In [375]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
import string 
import operator
from collections import Counter
import sklearn
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import sent_tokenize, word_tokenize, TweetTokenizer 
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from porter2stemmer import Porter2Stemmer
import numpy as np
from scipy import spatial

In [376]:
from autocorrect import spell

In [377]:
stemmer = Porter2Stemmer()
lemmatizer = WordNetLemmatizer()
from nltk.corpus import wordnet

word_tokenizer = word_tokenize
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
from nltk.corpus import stopwords

def get_wordnet_pos(word): 
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)
def lemmatize(text):
    return ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)])

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]    
    
def normalize(text):
    text = lemmatize(text)
    stop = set(stopwords.words('english'))
    tokens = word_tokenizer(text.lower().translate(remove_punctuation_map))
#     tokens = stem_tokens([spell(token) for token in tokens])
    return ' '.join([token for token in tokens if token not in stop])

In [378]:
from gensim.models import KeyedVectors
filename = 'GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(filename, binary=True)
index2word_set = set(model.wv.index2word)

In [381]:
def similarity_scores(text1, text2):
    similarity = model.wv.n_similarity(normalize(text1).split(), normalize(text2).split())
    return similarity

In [172]:
def similarities_int_array(arr1,arr2):
    intersection = list(set(arr1) & set(arr2))
    try:
        if len(set(arr1)) > len(set(arr2)):
            score = len(intersection)/len(set(arr1))
        score = len(intersection)/len(set(arr2))
    except:
        score = 0
    return score

def separate(text):
    import re  
    ints = re.findall('\\d+', text)
    word_tokenize(text)
    l3 = [x for x in word_tokenize(text) if x not in ints]
    return ' '.join(l3),ints

def final_similarity(text1,text2):
    text_part_1 , int_array1 = separate(normalize(text1))
    text_part_2 , int_array2 = separate(normalize(text2))
    sim_int_arrys = similarities_int_array(int_array1,int_array2)
    sim_text_part = similarity_scores(text_part_1,text_part_2)
    if len(list(int_array1+int_array2)) == 0:
        return sim_text_part
    else:            
        return 0.9*sim_text_part + 0.1*sim_int_arrys

In [173]:
def similarity_matrix(list_of_descriptions, list_of_pl):
    main = []
    for i in range(len(list_of_descriptions)):
        ary = []
        for j in range(len(list_of_descriptions)):
            ary.append(final_similarity(list_of_descriptions[i],list_of_descriptions[j]))
        main.append(ary)
    similarity_matrix = pd.DataFrame(main)
    similarity_matrix.columns = list_of_descriptions
    similarity_matrix.index = list_of_pl
    return similarity_matrix

In [382]:
dataframe = similarity_matrix(['hexagonal head bolts & nuts ', 'hexagonal head bolt with thread ','bracket fan','visit chair','supreme sofa','L ANGLE FOR MAIN LINE BRAKE PIPE FITTINGS TYPE 4', 'L ANGLE FOR MAIN LINE BRAKE PIPE FITTINGS TYPE 3'], [73030685,73031781,45181019,'7831N003','7831N002','3098NS', '3098NS'])

In [384]:
similarity_matrix(list_of_descriptions, list_of_pl)

,caring is sharing and will be always 1.,caring is sharing and 23 will be always.,caring is sharing and will be always.,caring is sharing and will be always.,caring is sharing and will be always in the world.,caring is sharing and will not be always.,caring is sharing and will always.,caring wass shared and shall be always shared.,caring is sharing
1567,1.000000,0.900000,0.900000,0.900000,0.823111,0.900000,0.900000,0.722604,0.806004
562,0.900000,1.000000,0.900000,0.900000,0.823111,0.900000,0.900000,0.722604,0.806004
334567,0.900000,0.900000,1.000000,1.000000,0.914568,1.000000,1.000000,0.802893,0.895560
476,0.900000,0.900000,1.000000,1.000000,0.914568,1.000000,1.000000,0.802893,0.895560
45675,0.823111,0.823111,0.914568,0.914568,1.000000,0.914568,0.914568,0.756287,0.803178
3456,0.900000,0.900000,1.000000,1.000000,0.914568,1.000000,1.000000,0.802893,0.895560
734567,0.900000,0.900000,1.000000,1.000000,0.914568,1.000000,1.000000,0.802893,0.895560
845678,0.722604,0.722604,0.802893,0.802893,0.756287,0.802893,0.802893,1.000000,0.748966
3456789,0.806004,0.806004,0.895560,0.895560,0.803178,0.895560,0.895560,0.748966,1.000000


In [187]:
list_of_pl = [1567,562,334567,476,45675,3456,734567,845678,3456789]
list_of_descriptions = ["caring is sharing and will be always 1.","caring is sharing and 23 will be always.","caring is sharing and will be always.","caring is sharing and will be always.","caring is sharing and will be always in the world.","caring is sharing and will not be always.","caring is sharing and will always.","caring wass shared and shall be always shared.", "caring is sharing"]

In [188]:
def conv7to8(pl_7):
    if len(str(pl_7)) != 7:
        return ("Number length not equal to 7")
    copy = pl_7
    sum = 0
    for i in range(2,9):
        sum = sum + copy%10*i
        #print(sum)
        copy = copy//10

    result = sum%11
    #print(result)
    z = int(str(pl_7) + str(result))
    return(z)

def conv4to7to8(pl_smaller_7):
    ary = [int(i) for i in str(pl_smaller_7)]
    if len(ary)>=7:
        return "Number length greater than 7"
    while len(ary)<7:
        from random import randint
        ary.append(randint(0, 9))
    #print(ary)
    final = ''
    for i in ary:
        final = final+str(i)
    #print(final)
    return conv7to8(int(final))

def compare(ass_pl, tupl):
    temp1 = int(str(ass_pl)[:4])
    #print(temp1)
    temp2 = int(str(tupl[1])[:4])
    #print(temp2)
    if str(temp1)[:2] != str(temp2)[:2]:
        pl = conv4to7to8(temp1)
    else :
        pl = conv4to7to8(temp2)
        
    return (pl,tupl[0])

In [189]:
sim = similarity_matrix(list_of_descriptions, list_of_pl)

In [190]:
sim

,caring is sharing and will be always 1.,caring is sharing and 23 will be always.,caring is sharing and will be always.,caring is sharing and will be always.,caring is sharing and will be always in the world.,caring is sharing and will not be always.,caring is sharing and will always.,caring wass shared and shall be always shared.,caring is sharing
1567,1.000000,0.900000,0.900000,0.900000,0.823111,0.900000,0.900000,0.722604,0.806004
562,0.900000,1.000000,0.900000,0.900000,0.823111,0.900000,0.900000,0.722604,0.806004
334567,0.900000,0.900000,1.000000,1.000000,0.914568,1.000000,1.000000,0.802893,0.895560
476,0.900000,0.900000,1.000000,1.000000,0.914568,1.000000,1.000000,0.802893,0.895560
45675,0.823111,0.823111,0.914568,0.914568,1.000000,0.914568,0.914568,0.756287,0.803178
3456,0.900000,0.900000,1.000000,1.000000,0.914568,1.000000,1.000000,0.802893,0.895560
734567,0.900000,0.900000,1.000000,1.000000,0.914568,1.000000,1.000000,0.802893,0.895560
845678,0.722604,0.722604,0.802893,0.802893,0.756287,0.802893,0.802893,1.000000,0.748966
3456789,0.806004,0.806004,0.895560,0.895560,0.803178,0.895560,0.895560,0.748966,1.000000


In [385]:
def conv7to8(pl_7):
    if len(str(pl_7)) != 7:
        return ("Number length not equal to 7")
    copy = pl_7
    sum = 0
    for i in range(2,9):
        sum = sum + copy%10*i
        #print(sum)
        copy = copy//10

    result = sum%11
    #print(result)
    z = int(str(pl_7) + str(result))
    return(z)

def conv4to7to8(pl_smaller_7):
    ary = [int(i) for i in str(pl_smaller_7)]
    if len(ary)>=7:
        return "Number length greater than 7"
    while len(ary)<7:
        from random import randint
        ary.append(randint(0, 9))
    #print(ary)
    final = ''
    for i in ary:
        final = final+str(i)
    #print(final)
    return conv7to8(int(final))

def return_dummy_pl(pl1, pl2, sim_score):
    
    if str(pl1)[:1] != str(pl2)[:1]:
        
        if sim_score>0.7 and sim_score<0.9:
            nw_pl = str(pl1)[:4] 
            nw_pl = conv4to7to8(nw_pl)
        
        elif sim_score>=0.9:
            nw_pl = str(pl1)[:6] 
            nw_pl = conv4to7to8(nw_pl)
    
    
    elif str(pl1)[:1] == str(pl2)[:1]:
        
        if str(pl1)[2:4] == str(pl2)[2:4]:
            if sim_score>0.7 and sim_score<0.9:
                nw_pl = pl2 
                #nw_pl = conv4to7to8(nw_pl)
            elif sim_score>=0.9:
                nw_pl = pl2 
                #nw_pl = conv4to7to8(nw_pl)
        
        elif str(pl1)[2:4] != str(pl2)[2:4]:
            if sim_score>0.7 and sim_score<0.9:
                nw_pl = pl2 
                #nw_pl = conv4to7to8(nw_pl)
            elif sim_score>=0.9:
                nw_pl = str(pl1)[:6] 
                nw_pl = conv4to7to8(nw_pl)
            
    
    return nw_pl  

In [386]:
import random
import numpy as np
import pandas as pd

In [464]:
low_threshold=0.7
final_dict={}
while(len(df)>1):
    temp_dict={}
    master_PL = df.index.values[0]
    master_Desc = df.columns.values[0]
    temp_dict[master_PL]=master_Desc
    final_dict.update(temp_dict)
    second_max_array = np.sort(df.values)[:,-2]
    delete_list=[]
    for j in range(len(df)):
        if df.iloc[0][j]>low_threshold:
            if second_max_array[j] == df.iloc[0][j]:
                delete_list.append(j)
                new_pl = return_dummy_pl(master_PL,df.index.values[j],df.iloc[0][j])
                tempdi = {}
                tempdi[new_pl]=df.columns.values[j]
                final_dict.update(tempdi)
            elif str(master_PL)[:4]==str(df.index.values[j])[:4]:
                delete_list.append(j)
                new_pl = return_dummy_pl(master_PL,df.index.values[j],df.iloc[0][j])
                tempdi = {}
                tempdi[new_pl]=df.columns.values[j]
                final_dict.update(tempdi)
                
    delete_list.sort(reverse=True)
    for each in delete_list:
        try:
            df = df.drop(df.index[each])
            df = df.drop([df.columns.values[each]],axis=1)
        except:
            pass
temp_dict[master_PL]=master_Desc
final_dict.update(temp_dict)

In [465]:
final_dict

{73030685: 'hexagonal head bolts & nuts ',
 73031781: 'hexagonal head bolt with thread ',
 45181019: 'bracket fan',
 '7831N003': 'visit chair',
 '7831N002': 'supreme sofa',
 '3098NS': 'L ANGLE FOR MAIN LINE BRAKE PIPE FITTINGS TYPE 4'}